In [6]:
import ee
import pandas as pd

In [2]:
ee.Authenticate()

Enter verification code:  4/1AeaYSHDQLkALuF_BrIrCPsXaXmHwSQu9fkVeyVpiBnMd5OuQkSHvYAT22co



Successfully saved authorization token.


In [5]:
ee.Initialize()

In [7]:
stations = pd.read_csv("/home/khanalp/task1/data/stations.csv")

In [23]:
coordinates = stations.loc[0,['Position']]

In [31]:
# Extracting the coordinates from the Series
coordinate_str = coordinates.iloc[0]

# Splitting the string to get latitude and longitude
longitude, latitude = map(float, coordinate_str.split())

In [32]:
# Define the location (longitude, latitude)
point = ee.Geometry.Point(longitude, latitude)

In [34]:
point.getInfo()

{'type': 'Point', 'coordinates': [2.284102, 47.322918]}

In [35]:
# Define the date range
start_date = '2000-01-01'
end_date = '2020-12-31'

In [36]:
# Create a buffer of 1km around the point
buffered_point = point.buffer(500)

In [42]:
# Create an image collection for MODIS LAI data
modis_collection = ee.ImageCollection('MODIS/006/MOD15A2H') \
    .filterBounds(buffered_point) \
    .filterDate(ee.Date(start_date), ee.Date(end_date)) \
    .select(['Lai_500m','FparLai_QC','LaiStdDev_500m']) #Add other bands too

In [44]:
modis_collection.first().bandNames().getInfo()

['Lai_500m', 'FparLai_QC', 'LaiStdDev_500m']

In [43]:
# Check the number of images in the collection
print("Number of images:", modis_collection.size().getInfo())

Number of images: 957


In [46]:
# Define the export region
export_region = buffered_point.bounds()

In [50]:
# Function to export images
def export_image(image):
    # Get the timestamp from system:time_start property
    timestamp = ee.Date(image.get('system:time_start')).format('yyyyMMdd').getInfo()
    
    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(image=image,
                                         description='MODIS_LAI' + timestamp,
                                         folder='MODIS_LAI_Export',
                                         region=export_region,
                                         scale=500,
                                         crs='EPSG:4326')
    task.start()



In [51]:
# Export all images in the collection
image_list = modis_collection.toList(modis_collection.size())
for i in range(image_list.size().getInfo()):
    export_image(ee.Image(image_list.get(i)))

**Steps to do**
1) Read all tif files in the xarray.
2) Then, select the grid cell containing particular location and neighboring 8 grid cells.
3) Keep only good-quality data (QC flag values0, 2, 24, 26, 32, 34, 56, and 58) and set all other values to missing.
4) At each time step, calculate a weighted mean from the nine pixels by weighting them by their standard deviation error (defined as $1/\sigma^2$).
5)  Gap fill the resulting 8-daily time series using a cubic spline function (Forsythe et al., 1977) and set any negative LAI values set to zero.
6)  The gap-filled time series was first smoothed using a cubic smoothing spline.
7)  A climatology (46 time steps) was then calculated from all available years.
8)  An anomaly time series was then created by removing the climatology and smoothed by taking a rolling mean over a window of ±6 time steps to further remove short-term variability.
9)  The climatology was then added to the smoothed time series and the 8-daily time series interpolated to the time resolution of
the flux tower data using the climatological values prior to MODIS commencing in January 2000.